In [5]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model

In [6]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

In [7]:
from keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [8]:
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [9]:
IMAGE_SIZE = [128, 128]

In [10]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [11]:
for layer in vgg.layers:
  layer.trainable = False

In [12]:
x = Flatten()(vgg.output)
prediction = Dense(4, activation='softmax')(x)

In [13]:
prediction.shape

(None, 4)

In [14]:
# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

# view the structure of the model
model.summary()

# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 128, 128, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 128, 128, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 128, 128, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 64, 64, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 64, 64, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 64, 64, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 32, 32, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 32, 32, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 32, 32, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 16, 16, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 16, 16, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 16, 16, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 8, 8, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 8, 8, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 4)              │        32,772 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,747,460 (56.26 MB)

 Trainable params: 32,772 (128.02 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [15]:
import os

In [16]:
import cv2

In [17]:
def load_images(directory_path, target_size=(128, 128)):
    images = []
    for filename in os.listdir(directory_path):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            img_path = os.path.join(directory_path, filename)
            img = load_image(img_path, target_size=target_size)  
            images.append(img)
    return np.array(images)



def load_image(image_path, target_size=(128, 128)):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  
    img = cv2.resize(img, target_size)
    return img



In [18]:
covid_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//COVID//images'
opacity_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Lung_Opacity//images'
normal_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Normal//images'
pneumonia_images_dir = 'C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19//COVID-19_Radiography_Dataset//Viral Pneumonia//images'


covid_images = load_images(covid_images_dir)
opacity_images = load_images(opacity_images_dir)
normal_images = load_images(normal_images_dir)
pneumonia_images = load_images(pneumonia_images_dir)


X = np.concatenate([covid_images, opacity_images, normal_images, pneumonia_images], axis=0)

In [19]:
covid_label = 0
opacity_label = 1
normal_label = 2
pneumonia_label = 3

covid_labels = np.full((len(covid_images),), covid_label)
opacity_labels = np.full((len(opacity_images),), opacity_label)
normal_labels = np.full((len(normal_images),), normal_label)
pneumonia_labels = np.full((len(pneumonia_images),), pneumonia_label)

In [20]:
Y = np.concatenate([covid_labels, opacity_labels, normal_labels, pneumonia_labels], axis=0)


In [21]:
shuffle_index = np.random.permutation(len(X))
X_ = X[shuffle_index]
Y_ = Y[shuffle_index]

In [22]:
Y_.shape

(21165,)

In [23]:
X_.shape

(21165, 128, 128, 3)

In [24]:
from keras.utils import to_categorical

# Assuming Y_train is loaded or generated with integer labels
Y_ = to_categorical(Y_, num_classes=4)

In [25]:
Y_.shape

(21165, 4)

In [27]:
from sklearn.model_selection import train_test_split

In [28]:
X_train, X_test, Y_train, Y_test = train_test_split(X_, Y_, test_size=0.2, random_state=42)

In [29]:
Y_train.shape

(16932, 4)

In [30]:
X_train.shape

(16932, 128, 128, 3)

In [31]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
print(f"Y_train shape before: {Y_train.shape}") 
history = model.fit(X_train, Y_train, batch_size=16, epochs=10, validation_data=(X_test, Y_test))
train_loss, train_accuracy = model.evaluate(X_train, Y_train, verbose=0)
print(f"Training Accuracy: {train_accuracy * 100:.2f}%")

test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=0)
print(f"Testing Accuracy: {test_accuracy * 100:.2f}%")


Y_train shape before: (16932, 4)
Epoch 1/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 554s 522ms/step - accuracy: 0.7837 - loss: 4.1718 - val_accuracy: 0.8490 - val_loss: 3.5743
Epoch 2/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 559s 528ms/step - accuracy: 0.8825 - loss: 2.5696 - val_accuracy: 0.8687 - val_loss: 3.2821
Epoch 3/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 539s 509ms/step - accuracy: 0.9035 - loss: 1.9821 - val_accuracy: 0.8684 - val_loss: 5.2239
Epoch 4/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 487s 460ms/step - accuracy: 0.9179 - loss: 1.8831 - val_accuracy: 0.8429 - val_loss: 6.5067
Epoch 5/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 526s 497ms/step - accuracy: 0.9247 - loss: 1.6839 - val_accuracy: 0.8814 - val_loss: 5.1341
Epoch 6/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 529s 499ms/step - accuracy: 0.9379 - loss: 1.3961 - val_accuracy: 0.8623 - val_loss: 4.8873
Epoch 7/10
1059/1059 ━━━━━━━━━━━━━━━━━━━━ 506s 478ms/step - accuracy: 0.9381 - loss: 1.4213 - val_accuracy: 0.8847 - val_loss: 4.7104
Epoch 8/10
1059/1059 ━━━━━━━━

In [32]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from keras_tuner import HyperModel

class MyHyperModel(HyperModel):
    def __init__(self, base_model):
        self.base_model = base_model

    def build(self, hp):
        # Use the existing model
        base_model = self.base_model
        base_model.trainable = False  # Freeze the base model
        
        model = Sequential()
        model.add(base_model)
        model.add(Flatten())
        
        # Add a dense layer with tunable units and dropout
        model.add(Dense(units=hp.Int('units', min_value=32, max_value=512, step=32), activation='relu'))
        model.add(Dropout(rate=hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))
        
        # Output layer
        model.add(Dense(4, activation='softmax'))
        
        model.compile(optimizer='adam',
                      loss='categorical_crossentropy',
                      metrics=['accuracy'])
        
        return model


In [33]:

model_ = Sequential()
model_.add(model)
model_.add(Flatten())
model_.add(Dense(256, activation='relu'))
model_.add(Dropout(0.5))
model_.add(Dense(4, activation='softmax'))
model_.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [34]:
from sklearn.model_selection import KFold
import numpy as np
import keras_tuner as kt

kf = KFold(n_splits=5, shuffle=True, random_state=42)
val_scores = []

for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    Y_train_fold, Y_val_fold = Y_train[train_index], Y_train[val_index]

    tuner = kt.RandomSearch(
        MyHyperModel(model),
        objective='val_accuracy',
        max_trials=5,
        executions_per_trial=1,
        directory='C://Users//Shreyash Verma//Desktop//WORK//c//python//X_RAY_COVID-19',
        project_name='vgg16_tuning'
    )

    tuner.search(X_train_fold, Y_train_fold, epochs=10, validation_data=(X_val_fold, Y_val_fold))

    best_model = tuner.get_best_models(num_models=1)[0]
    val_loss, val_accuracy = best_model.evaluate(X_val_fold, Y_val_fold)
    val_scores.append(val_accuracy)

print(f'Cross-validation scores: {val_scores}')
print(f'Mean cross-validation accuracy: {np.mean(val_scores) * 100:.2f}%')


Trial 3 Complete [01h 31m 18s]
val_accuracy: 0.9583702683448792

Best val_accuracy So Far: 0.9583702683448792
Total elapsed time: 04h 11m 10s

Search: Running Trial #4

Value             |Best Value So Far |Hyperparameter
512               |128               |units
0.4               |0.4               |dropout

Epoch 1/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 602s 1s/step - accuracy: 0.9354 - loss: 0.0000e+00 - val_accuracy: 0.9581 - val_loss: 0.0000e+00
Epoch 2/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 595s 1s/step - accuracy: 0.9563 - loss: 0.0000e+00 - val_accuracy: 0.9581 - val_loss: 0.0000e+00
Epoch 3/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 596s 1s/step - accuracy: 0.9574 - loss: 0.0000e+00 - val_accuracy: 0.9581 - val_loss: 0.0000e+00
Epoch 4/10
424/424 ━━━━━━━━━━━━━━━━━━━━ 975s 2s/step - accuracy: 0.9544 - loss: 0.0000e+00 - val_accuracy: 0.9584 - val_loss: 0.0000e+00
Epoch 5/10
358/424 ━━━━━━━━━━━━━━━━━━━━ 35:35 32s/step - accuracy: 0.9576 - loss: 0.0000e+00

KeyboardInterrupt: 